### COROUTINES
Coroutines declared with the async/await syntax is the preferred way of writing asyncio applications. For example, the following snippet of code prints “hello”, waits 1 second, and then prints “world”:

In [1]:
import asyncio
import time

# Example 01
```python
async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

await main()
```

In [2]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

# Example 02
```python
# This example waits the first say_after execution that takes 1 second to complete,
# And after it's completed, execute the say_after second execution
# The whole process as it's stopping at each "sleep", takes around 3 seconds to complete.
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

await main()
```

# Example 02
This example leverages the asyncio create_task() method to run the tasks concurrently.
```python
async def main():
    task_1 = asyncio.create_task(say_after(1, 'hello'))
    task_2 = asyncio.create_task(say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take around 2 seconds), this works concurrently
    await task_1
    await task_2

    print(f"finished at {time.strftime('%X')}")
    
await main()
```

# Example 03
This example leverages the asyncio TaskGroup class, that provides an alternative and more modern to create_task() method. Using this, the previous example is implemented in the following way:
```python
async def main():
    async with asyncio.TaskGroup() as task_group:
        task_1 = task_group.create_task(
            say_after(1, 'hello'))
        task_2 = task_group.create_task(
            say_after(2, 'world'))

        print(f"started at {time.strftime('%X')}")

    # The await is implicit when the context manager exits.
    
    print(f"finished at {time.strftime('%X')}")
    
await main()
```

The timing and output should be the same as the example_02 version.

# Awaitables

We say that an object is an awaitable object if it can be used in an await expression. Many asyncio APIs are designed to accept awaitables.

There are three main types of awaitable objects: coroutines, Tasks, and Futures.

# Coroutines

Python coroutines are awaitables and therefore can be awaited from other coroutines:

In [3]:
async def nested():
    return 42

Running courotine functions, without awaiting and with awaiting diference between.
```python
async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*. 
    nested()

    #Let's do it differently now and await it:
    print(await nested())

await main()
```

In [4]:
# Using tasks to schedule the courotines concurrently.

# async def main():
#     # Schedule nested() to run soon concurrently
#     # with "main()".
#     task = asyncio.create_task(nested())

#     # "task" can now be used to cancel "nested()", or
#     # can simply be awaited to wait until it is complete:
#     print(await task)

# await main()

### Futures

A Future is a special low-level awaitable object that represents an eventual result of an asynchronous operation.

When a Future object is awaited it means that the coroutine will wait until the Future is resolved in some other place.

Future objects in asyncio are needed to allow callback-based code to be used with async/await.

Normally there is no need to create Future objects at the application level code.

Future objects, sometimes exposed by libraries and some asyncio APIs, can be awaited:

In [2]:
import concurrent.futures
import asyncio

def blocking_io():
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open('/dev/urandom', 'rb') as f:
        return f.read(100)
        
def cpu_bound():
    # CPU-bound operations will block the event loop:
    # in general it is preferable to run them in a
    # process pool.
    return sum(i * i for i in range(10 ** 7))

async def main():
    loop = asyncio.get_running_loop()

    ## Options:

    # 1. Run in the default loop's executor:
    result = await loop.run_in_executor(
        None, blocking_io
    )
    print(f'\n')
    print("default thread pool", result)
    
    # 2. Run in a custom thread pool:
    with concurrent.futures.ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool, blocking_io)
        print('custom thread pool', result)
    
    # 3. Run in a custom process pool:
    # with concurrent.futures.ProcessPoolExecutor(max_workers=4) as pool:
    #     result = await loop.run_in_executor(
    #         pool, cpu_bound)
    #     print('Custom process pool', result)

if __name__ == "__main__":
   await main()



default thread pool b'\x1e\x84\xaae\xe7 N\xc8\xcb\xc4n\xae\xf5\x83\xc4\xacY\xf6\xf0\x9f\x8d{\xc5dq4U\xf9\xbeId\x01\x9f\xac/\x17\xe2\xee\xc1k\xabt\x06\xe6b\xbds\xd2l\x14\x85\xa0@\xc3\xb9\xe9\x97\x1b\xde$\xc7?V\x9e/\r\x03R\xbd\x97\\Z\x8d\x82\x1eT\x8b\xf36\t3\x08xEL\x85azG7;\xa1\xa6\xaf\xd6\xe5\x7f\xcb\xd0\xc2'
custom thread pool b'\x1f\x16\xe7:\x82\xd8\x90\x80\x90\x0cx\xde\xd6\xed\x91\xe9\xf2\x9f\x90s_\xde\xa0:\x8a-\x07\xbc\xfaO[6_\\\x95j\xf4\xc2\x80\x98\xc78\xa87\x1a\xd5\x03u\xac\xe5\x93\xd5\xbcf\xc9\x9d\xa3\xb2\x06B$y\xe1\xdd\x95\x1b\x1anu\x9b!-\x16}\x04\x06\xd9)\xd5\xd3\x12h\xf8^M\x15<\xe9\xb7M\xf7\x97\xbc\x7f\xc2j\x8b\x932\xb1'


In [6]:
background_tasks = set()

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

for i in range(10):
    task = asyncio.create_task(say_after(i, f"Hello there N.{i}"))

    # Add task to the set. This creates a strong reference
    background_tasks.add(task)

    # To prevent keeping references to finished tasks forever,
    # make each task remove its own reference from the set after
    # completion:
    task.add_done_callback(background_tasks.discard)

Hello there N.0
Hello there N.1
Hello there N.2
Hello there N.3
Hello there N.4
Hello there N.5
Hello there N.6
Hello there N.7
Hello there N.8
Hello there N.9
